In [ ]:
import pandas as pd
import nltk
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Column names: Index(['type', 'text'], dtype='object')
Dataset size: 5559
Number of features: 2


In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/sms_spam.csv",encoding='latin-1')
df.head(5900)


dataset_size = len(df)
num_features = len(df.columns)

print("Dataset size:", dataset_size)
print("Number of features:", num_features)

df = df[['type','text']]
df.rename(columns={'text':'messages','type':'Label'},inplace=True)
df.head(5559)

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/spam_sms.csv",encoding='latin-1')
df.head(5900)


dataset_size = len(df)
num_features = len(df.columns)

print("Dataset size:", dataset_size)
print("Number of features:", num_features)

df = df[['v2','v1']]
df.rename(columns={'v2':'messages','v1':'Label'},inplace=True)
df.head(4457)

In [ ]:
df.isnull().sum()

In [ ]:
Stopwords = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z]',' ',text)
    #remove extra spaces
    text = re.sub(r'\s+',' ',text)
    #remove stopwords
    text = "".join (word for word in text.split() if word not in Stopwords)
    return text

In [ ]:
df['clean_text']= df['messages'].apply(clean_text)
df.head()

In [ ]:
X = df['clean_text']
y = df['Label']

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
def classify(model, X, y):
    # train test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                               ('tfidf',TfidfTransformer()),
                               ('clf', model)])
    pipeline_model.fit(x_train, y_train)
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)
    y_pred = pipeline_model.predict(x_test)
    print(classification_report(y_test, y_pred, zero_division=1))
    conf = confusion_matrix(y_pred, y_test)
    sns.heatmap(conf)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix

def classify_kfold(model, X, y):
    # k-fold cross-validation
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                               ('tfidf',TfidfTransformer()),
                               ('clf', model)])
    scores = cross_val_score(pipeline_model, X, y, cv=kfold)
    print('Accuracy:', scores.mean()*100)
    y_pred = cross_val_predict(pipeline_model, X, y, cv=kfold)
    print(classification_report(y, y_pred, zero_division=1))
    conf = confusion_matrix(y, y_pred)
    print(conf)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model, X, y)
classify_kfold(model, X, y)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classify(model, X, y)
classify_kfold(model, X, y)

In [ ]:
from sklearn.svm import SVC
model = SVC(C=3)
classify(model, X, y)
classify_kfold(model, X, y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model, X, y)
classify_kfold(model, X, y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
classify(model, X, y)
classify_kfold(model, X, y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
classify(model, X, y)
classify_kfold(model, X, y)